In [290]:
import pandas as pd
import numpy as np

In [278]:
class PCA: #X의 차원을 축소
    def __init__(self, n):
        self.n= n #n차원으로 줄인다.
    
    def fit(self, X):
        cov_mat = np.cov(X.T) #공분산 행렬 계산
        vals, vecs = np.linalg.eig(cov_mat) #고유벡터 및 고유값 계산
        
        sorted_idx = np.argsort(vals)[::-1] #고유값을 내림차순으로 정렬한다
        vals = vals[sorted_idx] #정렬된 인덱스를 적용
        vecs = vecs[:, sorted_idx]
        self.components = vecs[:, :self.n] #고유 벡터 중 주성분을 추출
    
    def transform(self, X):
        X_transformed = X.dot(self.components) #기존 X 데이터에 주성분을 내적하여 차원이 축소된 행렬을 구한다
        return X_transformed
    

In [378]:
train = pd.read_csv("train_none_index.csv")
sub_test = pd.read_csv('test.csv')

data = train.iloc[:,:11].to_numpy()
target = train['label'].to_numpy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, random_state=123) # 데이터 분할


PCA를 통해 데이터의 차원 축소

In [364]:
pca = PCA(n_components=7)
pca.fit(X_train)
pca.fit(X_test)
pca.fit(sub_test)

# 데이터 차원 축소
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
sub_test = pca.transform(sub_test)


In [327]:
X_train

array([[ 1.05859529e+03, -6.76189450e+01, -7.87845984e+01, ...,
        -6.10728651e+00, -1.78941621e+01, -3.32092063e+01],
       [ 1.79604608e+02,  1.72108134e+02,  2.75624681e+01, ...,
         9.77774172e+01, -4.06438961e+01,  1.22154263e+02],
       [ 2.19952935e+01, -1.25641089e+00, -1.37467460e+00, ...,
        -3.52253755e+00,  2.89964345e+00,  1.49707883e+00],
       ...,
       [ 2.10814427e+01, -6.97658536e-01, -2.53032294e+00, ...,
        -5.70203187e-01, -7.69159188e-01, -3.28231352e-01],
       [ 1.66443593e+03,  2.30960574e+02,  4.17235199e+01, ...,
        -1.20627819e+02, -1.41586097e+02, -1.00209483e+02],
       [ 2.61536653e+03,  4.46446194e+02,  1.63768106e+03, ...,
         6.87968008e+02, -2.07032532e+02, -2.39550060e+02]])

In [282]:
X_test.shape[0]

2

In [9]:
#from svm import SVMClassifier  # Support Vector Classifier
from sklearn.metrics import classification_report

In [351]:
class SVMClassifier:
    def __init__(self,n_iters=100, lr = 0.01, random_seed=3, lambda_param=0.01):
        self.n_iters = n_iters # 몇 회 반복하여 적절한 값을 찾을지 정하는 파라미터
        self.lr = lr  # 학습률과 관련된 파라미터 
        self.lambda_param = lambda_param
        self.random_seed = random_seed
        np.random.seed(self.random_seed)

    def fit(self, x, y):

        n_samples, n_features = x.shape

        #  y값을 SVM 계산에 활용해주기 위하여 0에 해당하는 y값들을 -1로 변환
        y_ =  np.where(y==0,-1,y)  #y==0이 true라면 -1로 변경
        
        # w값 초기화, (n_features, )의 크기를 가지는 0과 1사이의 랜덤한 변수 어레이
        init_w = np.random.rand(n_features)
        self.w = init_w
        self.b = 0 # b값 초기화

        for _ in range(self.n_iters):
            for i in range(n_samples):
                x_i = x[i]
                y_i = y_[i]
                condition = (y_i * (np.dot(self.w , x_i)) + self.b) >= 1 #y(i) * (w · x(i) + b) >= 1
                if condition: #gradient loss function 수식을 통한 업데이트
                    self.w -= self.lr * (2 * self.lambda_param * self.w)  #w = w - lr*(2*lambda*w)
                else:
                    self.w -= self.lr * -(y_i * x_i - 2 * self.lambda_param * self.w) #w = w + lr*(y_i*x_i-2*lambda*w)
                    self.b -= self.lr * y_i

        return self.w, self.b

    def predict(self, x):
        self.approximation = np.zeros(len(x)) #0으로 이루어진 넘파이 배열 생성
        for i, value in enumerate(x):
          self.app = np.dot(self.w,value) - self.b #approximation = W·X - b
          if self.app>=0: 
            output = 1
          else:
            output = 0
          self.approximation[i]=output #조건에 따라 approximation 배열에 대입

        return self.approximation

    def decision_function(self, X):  #OnevsRest에서 모델 간 확률 비교 위해 정의 / predict에서 class를 구분하는데 사용한 값/ 클 수록 해당 확률에 속할 가능성이 높다고 판단
      return np.dot(X,self.w) - self.b



# OnevsRest

In [352]:
class OneVsRest: #멀티클래스 분류를 위한 one vs rest 기법
    def __init__(self,n_iters=100, lr = 0.01, random_seed=3, lambda_param=0.01):
      self.n_class = 4
      self.n_iters = n_iters # 몇 회 반복하여 적절한 값을 찾을지 정하는 파라미터
      self.lr = lr  # 학습률과 관련된 파라미터 
      self.lambda_param = lambda_param

    def one_hot(self, y_train): #Y를 one_hot encoding하여 여러 개의 데이터셋 생성 ex) y_train[0] = 0번 라벨을 가지는 데이터는 1, 다른 데이터는 0으로 처리함 
        y_train = pd.get_dummies(y_train)
        return y_train
    
    def fit(self, X_train, y_train):
        y_one_hot = self.one_hot(y_train)
        self.models = np.empty(self.n_class,dtype=object)

        for i in range(self.n_class): #class 개수만큼의 classifier 생성하여 그 class에 속하는지 알아냄
            model = SVMClassifier(n_iters=self.n_iters,lr=self.lr,lambda_param=self.lambda_param)
            model.fit(X_train, y_one_hot.iloc[:,i]) #각 행 별로 가져온다 i번쨰 행 -> one_hot의 주석 참고
            self.models[i] = model

    def predict(self, X_test):
        y_pred = []
        decisionf_list = [model.decision_function(X_test) for model in self.models] #각 X_test에 대한 classifier들의 decision function을 도출하여, 그 값이 가장 큰 인덱스를 반환
        y_pred = np.argmax(decisionf_list,axis=0) #즉 반환된 인덱스에 속한 가능성이 가장 크다는 것을 의미

        return y_pred

    def get_accuracy(self, y_true, y_pred): #정확도 계산 코드
        self.true = y_true 
        self.pred = y_pred

        jud = (self.true == self.pred)  #실제값과 예측값 비교 array -> True와 false로 이루어
        wright = len(jud[jud==True]) #실제값 == 예측값
        wrong = len(jud[jud==False]) #실제값 != 예측값

        self.acc = float(wright/len(self.true)) #accuracy = True값 개수/ 전체 데이터 수 
          
        return f'Accuracy: {self.acc:.5f}'
    
 

In [353]:
acc=[]

# SVM 모델 객체 생성
svm = SVMClassifier()

# 탐색할 하이퍼파라미터 정의
param = {
    'n_iters': [50, 100, 200], # 몇 회 반복하여 적절한 값을 찾을지 정하는 파라미터
    'lr': [0.0001, 0.001, 0.01, 0.05], # 학습률과 관련된 파라미터
    'lambda_param': [0.01, 0.1, 1] # 람다 파라미터
}


for i in param['n_iters']:
    for j in param['lr']:
        for k in param['lambda_param']:
            temp = [i,j,k]
            onevsrest = OneVsRest(n_iters=i,lr=j,lambda_param=k)
            onevsrest.fit(X_train, y_train)
            y_pred = onevsrest.predict(X_test)
            acc.append(f'{temp}={onevsrest.get_accuracy(y_test,y_pred)}')



In [356]:
acc

['[50, 0.0001, 0.01]=Accuracy: 0.79919',
 '[50, 0.0001, 0.1]=Accuracy: 0.85047',
 '[50, 0.0001, 1]=Accuracy: 0.85574',
 '[50, 0.001, 0.01]=Accuracy: 0.85861',
 '[50, 0.001, 0.1]=Accuracy: 0.85238',
 '[50, 0.001, 1]=Accuracy: 0.63575',
 '[50, 0.01, 0.01]=Accuracy: 0.82555',
 '[50, 0.01, 0.1]=Accuracy: 0.68224',
 '[50, 0.01, 1]=Accuracy: 0.65732',
 '[50, 0.05, 0.01]=Accuracy: 0.71076',
 '[50, 0.05, 0.1]=Accuracy: 0.65061',
 '[50, 0.05, 1]=Accuracy: 0.64079',
 '[100, 0.0001, 0.01]=Accuracy: 0.85047',
 '[100, 0.0001, 0.1]=Accuracy: 0.84687',
 '[100, 0.0001, 1]=Accuracy: 0.65133',
 '[100, 0.001, 0.01]=Accuracy: 0.85766',
 '[100, 0.001, 0.1]=Accuracy: 0.84663',
 '[100, 0.001, 1]=Accuracy: 0.84807',
 '[100, 0.01, 0.01]=Accuracy: 0.85262',
 '[100, 0.01, 0.1]=Accuracy: 0.85047',
 '[100, 0.01, 1]=Accuracy: 0.65780',
 '[100, 0.05, 0.01]=Accuracy: 0.84951',
 '[100, 0.05, 0.1]=Accuracy: 0.63503',
 '[100, 0.05, 1]=Accuracy: 0.63168',
 '[200, 0.0001, 0.01]=Accuracy: 0.79248',
 '[200, 0.0001, 0.1]=Acc

test dataset = 0.33 -> 0.85790
               0.4 -> 0.73507
               0.1 -> 0.63280 

In [379]:
onevsrest = OneVsRest(lr=0.0001,n_iters=200,lambda_param=1)
onevsrest.fit(X_train, y_train)
y_pred = onevsrest.predict(X_test)
onevsrest.get_accuracy(y_test,y_pred)

'Accuracy: 0.77154'

테스트 코드에 적용

In [372]:
y_pred_sub =  onevsrest.predict(sub_test)

In [373]:
y_pred_sub

array([0, 0, 0, ..., 0, 0, 0])

In [374]:
submission = pd.DataFrame({'id':[x for x in range(1,sub_test.shape[0]+1)],'type':y_pred_sub})

In [375]:
submission['type'].value_counts()

0    2202
2     640
1     281
3      17
Name: type, dtype: int64

In [376]:
submission.to_csv('2021312394이해민_sub11.csv',index=False)